<h1> <center> ANN using TensorFlow </center></h1>

In [158]:
#importing tensorflow for machine learning algorithm
import tensorflow as tf
import numpy as np

In [159]:
#extracting MNIST dataset 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/sathyanarayanan/data/", one_hot=True)

Extracting /home/sathyanarayanan/data/train-images-idx3-ubyte.gz
Extracting /home/sathyanarayanan/data/train-labels-idx1-ubyte.gz
Extracting /home/sathyanarayanan/data/t10k-images-idx3-ubyte.gz
Extracting /home/sathyanarayanan/data/t10k-labels-idx1-ubyte.gz


<p align> We have initialized Notice the x input layer is 784 nodes corresponding to the 28 x 28 (=784) pixels, and the y output layer is 10 nodes corresponding to the 10 possible digits.  the input to be supplied is a Python dictionary, with each key being the name of the placeholder that we are filling. The placeholder variable specifies the number of samples to be input.</p>

In [160]:
# Python optimisation variables
learning_rate = 0.1
epochs = 5
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

<p align>We create W1 and B1 variables to connect the input layer to the hidden layer. This neural network will have 250 nodes in the hidden layer, so the size of the weight tensor W1 is [784, 250]. TensorFlow has a replicated version of the numpy random normal function, which allows you to create a matrix of a given size populated with random samples drawn from a given distribution.  Likewise, we create W2 and b2 variables to connect the hidden layer to the output layer of the neural network.</p>

In [161]:
# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([784, 250], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([250]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([250, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

<p align> We execute the standard matrix multiplication of the weights (W1) by the input vector x and we add the bias b1. Next, we finalise the hidden_out operation by applying a rectified linear unit activation function to the matrix multiplication. </p>

In [162]:
# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out) 

<p align> Again we perform the weight multiplication with the output from the hidden layer (hidden_out) and add the bias, b2. In this case, we are going to use a softmax activation for the output layer.</p>

In [163]:
# now calculate the hidden layer output - in this case, let's use a softmax activated
# output layer
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

<p align> We are converting the output y_ to a clipped version, limited between 1e-10 to 0.999999.  This is to make sure that we never get a case where we have a log(0) operation occurring during training. Next we have to do cross entropy calculation. The cross-entropy is measuring how inefficient our predictions are for describing the truth. </p>

In [164]:
#clips the output within the range
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
# performing backward propogation
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                         + (1 - y) * tf.log(1 - y_clipped), axis=1))

<p align>Here we are using the gradient descent optimiser.  We initialize it with a learning rate, then specify what we want it to do. This function will then perform the gradient and the backpropagation </p>

In [165]:
# add an optimiser to perform gradient descent on cross entropy
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

<p align> The tf.argmaxfunction returns the index of the maximum value in a vector / tensor.  So, the correct_prediction operation returns a tensor of size (m x 1) of True and False values designating whether the neural network has correctly predicted the digit.  We then calculated the mean accuracy from this tensor – first we have to cast the type of the correct_prediction operation from a Boolean to a TensorFlow float in order to perform the reduce_mean operation.  Once we’ve done that, we now have an accuracy operation ready to assess the performance of our neural network. </p>

In [166]:
# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

<p> We are running the session now, with the training data. total_batch is used to calculate the number of batches to run through each training epoch. In the loop we initialise the avg_cost variable to keep track of the average cross_entropy cost for each epoch. batch_x,batch_y are randomly extracted from the dataset. </p>
<p align> Session takes a list of operations to be performed on the arguments. In our case,it performs optimiser and cross_entropy operations and it puts the output to c, which is used to calculate average cost.</p>
<p> after the training is completed, we are running the session on the test set and calculating the accuracy of the test data</p>
<p> We are repeating the steps of test data in validation data, except that we iterate for 5 times for kfold validation. we observe that we got improved accuracy results for k fold validation when comapared to test data.  </p>

In [167]:
validated_output=[]
# start the session
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _,c = sess.run([optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
            #print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
    print("Performance of testing set = {0:.3f}%".format(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})*100))
    
    for i in range(5):
        validated_out=sess.run(accuracy, feed_dict={x: mnist.validation.images, y: mnist.validation.labels})
        validated_output.append(validated_out)
    performance_validation= np.mean(validated_output)
    print("Performance of validation set =" +str(performance_validation*100) +"%")
   
        
    
    

Performance of testing set = 96.610%
Performance of validation set =96.7800021172%
